# Convert code to run in Leon French's java app

Leon French shared a java optimizer with us March 5, 2019 that is probably faster than our python code, but probably is not re-ranking probes the same way we are. This code will load selected dataframes, align their wellids and probes, and save them out as tsvs so Leon's optimizer can read them.

Spiro got instructions from Leon via email January 30, 2020 to run the optimizer and see if we can save some time running our optimizations. Those instructions informed the format of tsv files generated here.

In [1]:
""" Define selected files to convert. """

# This is an expression matrix unsplit after being SRS-adjusted.
# It has 58,692 probes and 3,704 wellids.
# It needs to be pruned down to 15,745 probes (The fornito selection) and 1280 wellids (all of the 1280 mappable to glasser parcels).
# Stored as a python pickle, it's 1.7GB.
expr_file_whole = "/data/cache/fornito-glasser-expression-srs.df"

# This is an expression matrix split into training set #202 after being SRS-adjusted.
# It has 15,745 probes (The fornito selection) and 640 wellids (half of the 1280 mappable to glasser parcels).
# Stored as a python pickle, it's 78MB.
expr_file_half = "/data/splits/sub-all_hem-A_samp-glasser_prob-fornito/batch-train00202/parcelby-wellid_splitby-wellid.srs.df"

# This is an expression matrix split into training set #202 after being SRS-adjusted.
# It has 15,745 probes (The fornito selection) and 640 wellids (half of the 1280 mappable to glasser parcels).
# Stored as a python pickle, it's 39MB.
expr_file_quarter = "/data/splits/sub-all_hem-A_samp-glasser_prob-fornito/batch-train00402/parcelby-wellid_splitby-wellid.srs.df"

# This is a connectivity-similarity matrix.
# It has 2,731 wellids on each axis, so most of those can be dropped to match the wellids in expression.
# Stored as a python pickle, it's 57MB.
connsim_file = "/data/conn/hcp_niftismooth_grandmean_sim.df"


In [2]:
""" Load the files and remove any non-comparable wellids. """

import pickle

df_expr_whole = pickle.load(open(expr_file_whole, "rb"))
df_expr_half = pickle.load(open(expr_file_half, "rb"))
df_expr_quarter = pickle.load(open(expr_file_quarter, "rb"))
df_connsim = pickle.load(open(connsim_file, "rb"))

# Figure out which wellids are common to both dataframes, in original df_expr order
wellids_whole = [id for id in df_expr_whole.columns if id in df_connsim.columns]
print("Found {:,} wellids common to both {:,} in expression whole and {:,} in connectivity similarity.".format(
    len(wellids_whole), len(df_expr_whole.columns), len(df_connsim.columns),
))

wellids_half = [id for id in df_expr_half.columns if id in df_connsim.columns]
print("Found {:,} wellids common to both {:,} in expression split-half and {:,} in connectivity similarity.".format(
    len(wellids_half), len(df_expr_half.columns), len(df_connsim.columns),
))

wellids_quarter = [id for id in df_expr_quarter.columns if id in df_connsim.columns]
print("Found {:,} wellids common to both {:,} in expression split-quarter and {:,} in connectivity similarity.".format(
    len(wellids_quarter), len(df_expr_quarter.columns), len(df_connsim.columns),
))

# Keep only the relevant (common to both) data.
df_expr_whole_pruned = df_expr_whole.loc[:, wellids_whole]
df_connsim_whole_pruned = df_connsim.loc[wellids_whole, wellids_whole]
print("Whole expression went from {} to {} and connectivity-similarity from {} to {}.".format(
    df_expr_whole.shape, df_expr_whole_pruned.shape, df_connsim.shape, df_connsim_whole_pruned.shape
))

df_expr_half_pruned = df_expr_half.loc[:, wellids_half]
df_connsim_half_pruned = df_connsim.loc[wellids_half, wellids_half]
print("Split-half expression went from {} to {} and connectivity-similarity from {} to {}.".format(
    df_expr_half.shape, df_expr_half_pruned.shape, df_connsim.shape, df_connsim_half_pruned.shape
))

df_expr_quarter_pruned = df_expr_quarter.loc[:, wellids_quarter]
df_connsim_quarter_pruned = df_connsim.loc[wellids_quarter, wellids_quarter]
print("Split quarter expression went from {} to {} and connectivity-similarity from {} to {}.".format(
    df_expr_quarter.shape, df_expr_quarter_pruned.shape, df_connsim.shape, df_connsim_quarter_pruned.shape
))


Found 1,139 wellids common to both 1,280 in expression whole and 2,731 in connectivity similarity.
Found 568 wellids common to both 640 in expression split-half and 2,731 in connectivity similarity.
Found 283 wellids common to both 320 in expression split-quarter and 2,731 in connectivity similarity.
Whole expression went from (15745, 1280) to (15745, 1139) and connectivity-similarity from (2731, 2731) to (1139, 1139).
Split-half expression went from (15745, 640) to (15745, 568) and connectivity-similarity from (2731, 2731) to (568, 568).
Split quarter expression went from (15745, 320) to (15745, 283) and connectivity-similarity from (2731, 2731) to (283, 283).


In [3]:
df_expr_whole.to_pickle("./ahba_expr_whole.df")


In [4]:
""" Write the matched set to java-friendly tsvs. """

# This file ends up 342.5MB (the entire AHBA matrix was over 3GB)
df_expr_whole_pruned.index.name = ""
df_expr_whole_pruned.to_csv("./ahba_expr_whole.tsv", sep="\t")

df_connsim_whole_pruned.index.name = ""
df_connsim_whole_pruned.to_csv("./hcp_conn_sim_whole.tsv", sep="\t")

df_expr_half_pruned.index.name = ""
df_expr_half_pruned.to_csv("./ahba_expr_train202.tsv", sep="\t")

df_connsim_half_pruned.index.name = ""
df_connsim_half_pruned.to_csv("./hcp_conn_sim_train202.tsv", sep="\t")

df_expr_quarter_pruned.index.name = ""
df_expr_quarter_pruned.to_csv("./ahba_expr_train402.tsv", sep="\t")

df_connsim_quarter_pruned.index.name = ""
df_connsim_quarter_pruned.to_csv("./hcp_conn_sim_train402.tsv", sep="\t")


In [5]:
df_expr_quarter_pruned.to_pickle("./ahba_expr_train402.df")
df_connsim_quarter_pruned.to_pickle("./hcp_conn_sim_train402.df")

----

## For parcellated data, run separately

----

In [6]:
""" Load unparcellated data, then parcellate it. """

from pygest.rawdata.glasser import glasser_parcel_map
import pandas as pd


def average_expr_per_parcel(wellid_expression, parcel_map):
    """ Average expression values over all wellids in each parcel_map-defined parcel.
        (copied from pygest split.run and modified)
    """

    parcels = pd.DataFrame(
        data={'parcel': [parcel_map[x] for x in wellid_expression.columns]},
        index=wellid_expression.columns
    )
    parcel_means = {}
    for parcel in sorted(list(set(parcel_map.values()))):
        parcel_idx = parcels[parcels['parcel'] == parcel].index
        if len(parcel_idx) > 0:
            parcel_means[parcel] = wellid_expression.loc[:, parcel_idx].mean(axis=1)
        else:
            print("  parcel {} had no samples".format(parcel))
    return pd.DataFrame(data=parcel_means)

# SRS-adjusted expression is already loaded, by wellid, as df_expr_whole, from expr_file_whole
df_expr_parcellated = average_expr_per_parcel(df_expr_whole, glasser_parcel_map)

print("Full unparcellated dataframe [{:,} x {:,}] reduced to [{:,} x [{:,}].".format(
    df_expr_whole.shape[0], df_expr_whole.shape[1], df_expr_parcellated.shape[0], df_expr_parcellated.shape[1],
))


Full unparcellated dataframe [15,745 x 1,280] reduced to [15,745 x [177].


In [7]:
""" Find agreement between parcellated expresison and connectivity, then save out matching pair. """

# This is a parcellated connectivity-similarity matrix.
# It has 360 parcels on each axis, so half of those can be dropped to match the left hemisphere wellids in expression.
# Stored as a python pickle, it's 1MB.
parcellated_connsim_file = "/data/conn/glasser-connectivity_sim.df"

df_connsim_parcellated = pickle.load(open(parcellated_connsim_file, "rb"))

matching_parcels = [id for id in df_expr_parcellated.columns if id in df_connsim_parcellated.columns]
print("{:,} parcels are in both expression and connectivity-similarity.".format(len(matching_parcels)))

df_expr_parcellated_pruned = df_expr_parcellated[matching_parcels]
df_connsim_parcellated_pruned = df_connsim_parcellated.reindex(matching_parcels)[matching_parcels]
print("Parcellated expression went from {} to {} and parcellated connectivity-similarity from {} to {}.".format(
    df_expr_parcellated.shape, df_expr_parcellated_pruned.shape, df_connsim_parcellated.shape, df_connsim_parcellated_pruned.shape
))

df_expr_parcellated_pruned.to_csv("./ahba_expr_whole_parby-glasser.tsv", sep="\t")
df_expr_parcellated_pruned.to_pickle("./ahba_expr_whole_parby-glasser.df")
df_connsim_parcellated_pruned.to_csv("./hcp_conn_sim_whole_parby-glasser.tsv", sep="\t")
df_connsim_parcellated_pruned.to_pickle("./hcp_conn_sim_whole_parby-glasser.df")


177 parcels are in both expression and connectivity-similarity.
Parcellated expression went from (15745, 177) to (15745, 177) and parcellated connectivity-similarity from (360, 360) to (177, 177).
